## Uczenie głębokie z wykorzystaniem dedykowanych narzędzi

### Spis treści:
    1) Wstęp: oprogramowanie dedykowane sieciom neuronowym
    2) Wstep do obliczeń symbolicznych
        2.0) teoria
        2.1) ćwiczenie
    3) Keras
        3.0) prezentacja framework'u
        3.1) ćwiczenie: implementacja modelu
        3.2) implementacja warstw
        3.3) ćwiczenie: implementacja własnej warstwy
    4.) Hands-on computer vision: prezentacja i ćwiczenie z rozponawania obrazu

----

## 1.)  Wstęp: oprogramowanie dedykowane sieciom neuronowym

- Wraz z rosnącym zainteresowaniem sieciami neuronowymi, rosło zapotrzebowanie na dedykowane narzędzia, które ułatwiłyby i przyspieszyły proces tworzenia, a następnie uczenia głębokich modeli. 


- Z upływem lat kolejne grupy badawcze prezentowały swoje rozwiązania, a w ostatnim czasie tematem zainteresował się również przemysł. Dzięki nowym źródłom finansowania i niesłabnącemu zapotrzebowaniu, większość frameworków jest obecnie aktywnie rozwijana i ulepszana.

- Do najpopularniejszych narzędzi należą:

    - [Tensorflow](http://tensorflow.org) (**Google**)
        - Python, Go, C++
        - Najpopularniejszy z obecnie istniejących frameworków
        - Najintensywniej rozwijany

    - [Theano](http://deeplearning.net/software/theano/) (**U. Montreal**)
        - Python
        - Był prekursorem TensorFlow
        - Rozwijany przez uniwersytet w Montrealu (non-profit)
        - Obecnie coraz częściej porzucany na rzecz TF

    - [Torch](http://torch.ch/) (**Facebook, Twitter**)
        - Lua
        - Najmniej popularny ze względu na brak bindingów Pythonoych
        - Nie wspiera automatycznego różniczkowania

    - [MXNet](http://mxnet.io/) (**Amazon**)
        - Python, C++, Go, Julia, Scala, R, ...
        - Najwydajniejszy pod względem pamięci 
        - Od niedawna wspierany przez Amazon

<img style="float: left;" src="img/tf-logo-2.png"> 
<img style="float: left;" src="img/th-logo.png">
<img style="float: left;" src="img/torch-logo-f.png">
<img style="float: left;" src="img/mx-logo.png">

- Wszystkie z tych narzędzi są rozwijane jako proejkty OpenSource:

    - https://github.com/tensorflow/tensorflow

    - https://github.com/torch/torch7

    - https://github.com/Theano/Theano

    - https://github.com/dmlc/mxnet

- Wymienione wyżej narzędzia to zazwyczaj biblioteki operujące na dosyć niskim poziomie abstrakcji.

- Wokół nich powstało wiele projektów mających jeszcze bardziej ułatwić użytkownikom uczenie sieci neuronowych.

- Najpopularniejszym z takich projektów jest obecnie **Keras**, na którym skupimy się w dalszej części

![k](img/keras-logo.png)

----

## 2.) Wstep do obliczeń symbolicznych

- Sercem większości frameworków wysokiego poziomu jest jeden z wyżej wymienionych silników.
- Keras, którego będziemy dzisiaj używać, posiada dwa takie backendy: Tensorflow oraz Theano
- Warto poświęcić chwilę, żeby zapoznać się z najbardziej podstawowymi zastadami ich działania.

- Tensorflow i Theano to przykłady narzędzi, które tworzą graf operacji symbolicznych.

- Oznacza to, że operacje wykonywane na zmiennych nie mają natychmiastowego efektu. Są jedynie dodawane do grafu.

- Następnie graf jest kompilowany i wykonwyany.

Przykład:

In [28]:
import tensorflow as tf
tf.reset_default_graph()

a = tf.Variable(2.)
b = tf.Variable(2.)
result = a + b

print(result)

Tensor("add:0", shape=(), dtype=float32)


`result` nie jest równe 4! 

Jest tylko węzłem w grafie, symbolizującym operację dodania zmiennej `a` do `b`

In [29]:
show_graph(width=900, height=300)

Zbudowany graf możemy "wykonać", jako parametr `outptus` podając te zmienne, które chcemy obliczyć.

In [30]:
from helpers3 import execute_tf_graph

execute_tf_graph(result)

[4.0]

Dobrze, ale co zrobić, jeżeli chcemy obliczyć sumę `2 + 3`. Czy musimy zbudować i skompilować cały graf od nowa?

Na szczęście nie -- możemy podać wartości dowolnej zmiennej w grafie przy jego wywołaniu.

Aby to zrobić, tworzymy słownik, który przypisze wybranym zmiennym odpowiednie wartości

In [31]:
inputs = {
    a: 2,
    b: 3
}

execute_tf_graph(outputs=result, inputs=inputs)

[5.0]

Zwróć uwagę, że w słowniku tym nie podajemy nazw zmiennych (typu `str`), ale obiekty `Pythona`!

**Uwaga!**: w praktyce, jeżeli chcemy, żeby nasza zmienna była inicjalizowana dopiero przy wywołaniu grafu, 
oraz żeby jej podanie było obowiązkowe, należy **tf.Variable** zamienić na **tf.placeholder**

In [48]:
tf.reset_default_graph()
a = tf.placeholder(dtype=np.float32, name='zmienna_a')
result = a + 1

try:
    execute_tf_graph(result)
except tf.python.errors.InvalidArgumentError as e:
    print(e.message)

You must feed a value for placeholder tensor 'zmienna_a' with dtype float
	 [[Node: zmienna_a = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]


----

### zadanie 1: 
    - oblicz kwadraty liczb 2, 3 i 4 używając tensorflow. Wykorzystaj mechanizm placeholderów

### rozwiązanie 1: 

In [50]:
x = tf.placeholder(np.float32)
result = x * x

for i in {2, 3, 4}:
    outputs = result
    inputs = {
        x: i
    }
    
    returned = execute_tf_graph(outputs=result, inputs=inputs)
    print(returned)

[4.0]
[9.0]
[16.0]


### zadanie 2: 
    - dodaj dwie macierze jednostkowe 2x2

### rozwiązanie 2: 

In [53]:
A = tf.placeholder(dtype=np.float32)
B = tf.placeholder(dtype=np.float32)
result = A + B

inputs = {
    A: np.array([
            [1, 0],
            [0, 1]
        ]),
    B: np.array([
            [1, 0],
            [0, 1]
        ])
}

execute_tf_graph(outputs=[result], inputs=inputs)

[array([[ 2.,  0.],
        [ 0.,  2.]], dtype=float32)]

- Warto zaznaczyć, iż **Tensorflow został stworzony dla operacji na macierzach (tensorach)**. Radzi więc sobie wspaniale z wielowymiarowymi danymi. (Patrz rozw. powyżej)

- Jakie są jednak konkretne zalety tego symbolicznego podejścia? Trzy najważniejsze to:
    1. Optymalizacja: znając cały graf, kompilator może zoptymalizować wykonywane operacje
    2. Współbieżność: kompilator sam zadba o to, aby wykonać obliczenia równolegle
    3. Niezależność od architektury: Znając graf, kompilator może wygenerować kod dla CPU / GPU / FPGA etc.
    
- Szczególnie punkt trzeci jest tak istotny, gdyż obecnie do uczenia sieci neuronowych niemalże niezbędny jest procesor graficzny wspierający technologię `CUDA`

---- 

## 3.) Keras

In [17]:
# Wyższy poziom abstrakcji

# Warto wiedzieć, jakie są ogólne zasady rządzące tak popularnymi narzędziami jak tensorflow 
# W praktyce jednak, najczęściej korzystamy z narzędzi wyższego poziomu, które pozwalają nie myśleć o obliczeniach symbolicznych.

In [18]:
# Keras
    # Jednym z popularniejszych narzędzi jest Keras. 
    # Pozwala on łatwo tworzyć modele z popularnych "klocków"

In [23]:
# Aby z powodzeniem zastosować algorytm uczenia głębokiego 
# Potrzebne nam będą 4 elementy:
    # dane
    # architektura sieci
    # funckja straty
    # algorytm optymalizacji

In [ ]:
# Dane

# keras współpracuje z numpy, w związku z tym te etap nie będzie dla nas problemem
# wystarczy wczytać dane znane już z poprzednich zajęć:
X = np.load(...)
y = np.load(...)

# warto zaznaczyć, iż keras potrafi współpracować także z danymi w innym formacie, np. hdf5

In [ ]:
# Architektura sieci. 

# Keras wspiera dwie metody definiowania architektury: model liniowy i API funkcjonalne.

In [ ]:
# [pozbyć się tego kodu...?]

# Przykładowy model może wyglądać następująco:

# Liniowo:
model = Sequential()
model.add(Dense(32, input_dim=128))
model.add(Activation('relu'))

# Funckojnalnie:
x = Input(input_dim=128)
x = Dense(32)(x)
x = Activation('relu')(x)
model = Model(x)

In [26]:
# Skupimy się dzisiaj głównie na modelach liniowych, ponieważ w zupełności wystarczają one do większośći zastosowań praktycznych.

In [ ]:
# Warstwy:

# Podstawowym elementem, z którego budowana jest sieć neuronowa, jest warstwa. 
# Model liniowy w kerasie to nic innego, jak tylko złożenie kolejnych, następujących po sobie warstw.

# Warstwa (layer) jest podstawową jednostką przetwarzania: 
# przyjmuje ona jakąś macierz (tensor) na wejściu, modyfikuje ją, a następnie podaje na wyściu

In [ ]:
# Aby zobrazować to w praktyce, zaimplementujemy regresję logistyczną, poznaną na poprzednich zajęciach, z użyciem kerasa

In [ ]:
# Regresja logisytczna 

In [ ]:
# ćwiczenie?

In [ ]:
# MLP

In [ ]:
# ćwiczenie?

In [ ]:
# Implementacja własnej warstwy

In [ ]:
# ćwiczenie?

In [ ]:
# Teraz, kiedy znamy już podstawowe zasady rządzące Kerasem
# Pokażemy, w jak łatwy sposób można wykorzystać to narzędzie to rozpoznawania obrazu.

# W sposób trywialnie prosty otrzymamy wynik, który jeszcze parę lat temu pozostawał poza zasięgiem najbardziej zaawansowanych metod.

In [9]:
o.name# Budowanie modelu

'Variable/initial_value'

In [4]:
# Implementacja warstw

In [5]:
# Sieci konwolucyjne

In [7]:
# Potęga Deep Learning

# zbuduj swój model, mając "core"